In [2]:
import os
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
data_dir = '/Users/abdulkadirgokce/Documents/MSFeCare/MSF-eCARE-filedrop/'

!ls $data_dir

clean_data.csv                          metadata_patient.json
datadictionary-workinprogress.xlsx      ml-dataset-datadictionary_20211001.xlsx
full-310k - Copy xl.xlsx                patient_data.csv
full-310k - Copy.csv                    processed_data.csv
metadata.json                           ~$datadictionary-workinprogress.xlsx
metadata.pkl                            ~$full-310k - Copy xl.xlsx


In [4]:
filename = 'full-310k - Copy.csv'
file_dir = os.path.join(data_dir, filename)
df = pd.read_csv(file_dir)
df

/Users/abdulkadirgokce/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (0,2,5,8,9,43,46,48,51,54,114,129,130,131,134,135,136,154,156,158,160,182,183,184,186,187,191,202,203,204,208,212,234,248,252,265,266) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,source,project,hf,created,slot_id,a_age,a_gender2,a_weight,abdo_ds,abdo_inguin_mass,...,uti_vomiting,uvul_complication,uvulectomy,vacci_card,vaccines,vsd_added_signs,wfa,wfa2,wfa_less_than_neg3sd,wfh2
0,0,RCA-1,2,15feb2017 12:11:47,156.0,18.0,male,10.2,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,RCA-1,3,01mar2017 08:59:36,169.0,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,RCA-1,1,05jan2017 12:39:43,150.0,12.0,male,7.0,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,RCA-1,3,16aug2017 10:05:44,186.0,12.0,male,8.0,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,RCA-1,1,28apr2017 19:19:26,167.0,12.0,male,9.8,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
310299,8,RCA-2,801,12mar2020 09:55:48,158.0,37,female,12.6,NaN,0,...,0.0,0.0,0.0,0.0,notime,NaN,NaN,0.0,0.0,NaN
310300,8,RCA-2,801,08jan2020 08:26:21,158.0,5,male,7.2,NaN,0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,NaN
310301,8,RCA-2,801,04apr2020 10:06:07,158.0,26,female,12.1,[1fafe800-fd5b-11e3-a3ac-0800200c9a66],0,...,0.0,0.0,0.0,0.0,notime,NaN,NaN,0.0,0.0,NaN
310302,8,RCA-2,801,16jul2020 14:42:21,158.0,17,male,10.0,NaN,0,...,0.0,0.0,0.0,1.0,incomplete,NaN,NaN,0.0,0.0,NaN


In [4]:
def find_features(df, prefixes):
    """
    Return a list features starting with given prefixes
    """
    columns = df.columns
    found = []
    count = 0
    
    for prefix in prefixes:
        for c in sorted(columns):
            if c.find(prefix, 0, len(prefix))!=-1:
                print(c)
                count +=1
                found.append(c)

    print('')
    return count, found

def print_info(df, features, drop_list):
    """
    Print statistics of features
    """
    for c in sorted(features):
        if c not in drop_list: print(c, df[c].unique())

    print('')
    for c in sorted(features):
        if c not in drop_list: print(c, df[c].isna().sum()/len(df))

    print('')
    for c in sorted(features):
        print(c, df[c].dtype)
        
    print('')
    for c in sorted(features):
        if c not in drop_list: print(f'\t{c}',f'\n{df[c].value_counts(dropna=0)}')
    """"         
    print('')
    for c in sorted(features):
        if c not in drop_list:
            modes = dict(df[c].value_counts(dropna=0))
            print(f'{modes[0]/(modes[0]+modes[1]):.3f}', f'{modes[0]}/{modes[1]}', c)
    """
            
def balanced_features(df, features, threshold):
    """
    Check the frequency of labels of features
    """
    balanced = []
    for c in sorted(features):
        modes = dict(df[c].value_counts(dropna=0))
        if modes[0]/(modes[0]+modes[1]) < threshold and modes[0]/(modes[0]+modes[1]) > (1-threshold): 
            print(f'{modes[0]/(modes[0]+modes[1]):.3f}', f'{modes[0]}/{modes[1]}', c)
            balanced.append(c)
    
    unbalanced = [c for c in features if c not in balanced]
    return balanced, unbalanced

In [5]:
columns = df.columns

In [5]:
#### Drop a few entries

df = df.drop(df.index[df['source'].isin(['abdominal', 'eau', 'chaud', 'sro', 'coartem'])])
df = df.drop(df.index[df['t_ab_recommended'].isna()])
df = df.drop(df.index[df['a_gender2'].isna()])

In [6]:
df.shape

(310289, 271)

In [7]:
df.loc[df['wfa2']==-1000, 'wfa2'] = np.nan

In [8]:
df = df.astype(
    {
        'hf': 'int64',
        'a_age': 'float64',
        'classify_diarrhoea_acute_without': 'float64',
        'classify_dysentery_complicated': 'float64',
        'classify_dysentery_non_compl': 'float64',
        'classify_ear_mastoiditis': 'float64',
        'classify_eye_conjunctivitis_vira': 'float64'
    }
)

In [9]:
keep_list =[
    'source',
    'project',
    'hf',
    'created',
    'a_age',
    'a_gender2',
    'slot_id',
    'a_weight',
    'classify_abdo_gastro',
    'classify_abdo_unclassified',
    'classify_cough_pneumonia',
    'classify_cough_urti',
    'classify_dehydration_none',
    'classify_diarrhoea_acute_without',
    'classify_ear_acute_infection',
    'classify_eye_bacterial_conjuncti',
    'classify_eye_conjunctivitis_vira',
    'classify_likely_viral_infection',
    'classify_malaria',
    'classify_malaria_severe',
    'classify_malaria_simple_new',
    'classify_malnut_mam',
    'classify_malnut_sam',
    'classify_moderate_acute_malnutri',
    'classify_mouth_oral_trush',
    'classify_sam_with_no_complicatio',
    'classify_skin_eczema_simple',
    'classify_skin_fungal',
    'classify_skin_scabies_simple',
    'classify_skin_tinea_capitis',
    'classify_skin_unclassified_or_vi',
    'classify_uti_febrile',
    'cough_assessed', 
    'cough_indrawing', 
    'cough_wheezing',
    'danger_sign_vomit_only', 
    'danger_sign_vomiting',
    'wfa2',
    't_ab_recommended',
    't_am_recommended',
    'treat_ab_prescribed_hw', # High corr with the next 3
    'treat_ab_prescribed_tab',
    'treat_am_prescribed_hw',
    'treat_am_prescribed_tab',
    'skin_assessed',
    's_muac',
    's_temp',
    's_abdopain',
    's_cough',
    's_diarr',
    's_earpain',
    's_eyepb',
    's_fever_his',
    's_fever_temp',
    's_mouthpb',
    's_none',
    's_skin',
    's_vomit',
    'hisdx_diarrhblood',
    'hisdx_diarrhwatery',
    'hisdx_lrti',
    'hisdx_malaria_simple',
    'hisdx_malnut',
    'hisdx_ot_eye',
    'hisdx_ot_fever_viral',
    'hisdx_ot_mouth',
    'hisdx_ot_skin',
    'hisdx_ot_uti',
    'hisdx_others',
    'hisdx_otitis',
    'hisdx_severe',
    'hisdx_urti',
    'hydration_assessed',
    'labs_mrdt_before',
    'lab_malaria_done',
    'lab_malaria_pos',
    'lab_malaria_rdt',
    'lab_malaria_test_any_done',
    'lab_malaria_test_any_positive',
    'lab_urine_done',
    'malaria_presumptive', 
    'malaria_rdt_done',
    'ear_pain_new',
    'eye_pus',
    'mouth_assessed', 
    'mouth_trush',
    'vacci_card',
    'vaccines',
    'no_symptom_classifying_reported', 
    'no_symptom_ecare_reported',
    'bloody_stool',
    'cs_duration',
    
    
]


In [10]:
imbalanced_list = [
    'classify_UTI_febrile',
    'classify_abdo_constipation',
    'classify_abdo_possible_strangula',
    'classify_abdo_resp_pain',
    'classify_abdo_risk_of_testicular',
    'classify_abdo_sickle_cell_crisis',
    'classify_anaphylaxis',
    'classify_anaphylaxis_non_severe',
    'classify_anaphylaxis_severe',
    'classify_anemia_non_severe',
    'classify_anemia_severe',
    'classify_cough_bronchiolitis',
    'classify_cough_persistent',
    'classify_cough_wheezing_episode',
    'classify_cough_wheezing_resistan',
    'classify_cystitis_acute',
    'classify_dehydration_complicated',
    'classify_dehydration_moderate',
    'classify_dehydration_severe_new',
    'classify_diarrhoea_acute_with_de',
    'classify_diarrhoea_persistent',
    'classify_dysentery',
    'classify_dysentery_complicated',
    'classify_dysentery_intussuscepti',
    'classify_dysentery_non_compl',
    'classify_ear_chronic_discharge',
    'classify_ear_mastoiditis',
    'classify_ear_unclassified',
    'classify_eye_possible_corneal_da',
    'classify_eye_unclassified',
    'classify_fever_persistent',
    'classify_likely_viral_oma',
    'classify_malnut_2ds_only__6m',
    'classify_malnut_complicated_6_59',
    'classify_malnut_complicated__6m_',
    'classify_malnut_complicated_all',
    'classify_measles',
    'classify_measles_non_compl',
    'classify_measles_with_complicati',
    'classify_mouth_lesion_benign',
    'classify_mouth_stomatitis',
    'classify_pharyngitis_viral_',
    'classify_possible_streptococcal_',
    'classify_possible_trachoma',
    'classify_severe_abdo_pain',
    'classify_severe_complicated_meas',
    'classify_severe_resp_illness',
    'classify_skin_abscess_multiple',
    'classify_skin_abscess_single',
    'classify_skin_cellulitis_febrile',
    'classify_skin_cellulitis_non_feb',
    'classify_skin_cellulitis_periorb',
    'classify_skin_chickenpox_infecte',
    'classify_skin_chickenpox_severe',
    'classify_skin_chickenpox_simple',
    'classify_skin_eczema_infected',
    'classify_skin_furoncle_complicat',
    'classify_skin_furoncle_simple',
    'classify_skin_herpes',
    'classify_skin_impetigo_extensive',
    'classify_skin_impetigo_localized',
    'classify_skin_molluscum',
    'classify_skin_scabies_infected',
    'classify_skin_ulcer_febrile_or_l',
    'classify_skin_urticaria',
    'classify_skin_zona',
    'classify_uti_upper',
    'classify_uvulectomy_complicated',
    'classify_very_severe_disease_mal',
    'cough_malaise_his',
    'cough_prolonged',
    'cough_wheez_rhinitis',
    'cough_wheez_salbu_response',
    'cough_wheeze_rash',
    'danger_sign_cry',
    'danger_sign_cyanosis',
    'danger_sign_drink',
    'danger_sign_grunting',
    'danger_sign_interaction',
    'danger_sign_jaundice',
    'danger_sign_neck',
    'danger_sign_pallor_severe',
    'danger_sign_tonus',
    'danger_sign_vomit_confirmed',
    'd_convuls_his',
    'd_convuls_now',
    'd_gends_any',
    'd_nobreath',
    'd_unconscious',
    'abdo_inguin_mass',
    'abdo_intense_testispain',
    'abdo_painkiller_response',
    'skin_chickenpox_ds',
    'skin_eye_swelling',
    'skin_pustule_ds',
    'skin_swelling_large',
    'skin_tender_mass',
    'skin_ulcerlarge',
    'skin_vesicles_ds',
    's_pallor',
    's_drepano',
    's_dysuria',
    's_hematuria',
    's_joint',
    's_limp',
    's_oedema',
    's_throat',
    'ms_measles',
    'hisdx_anemia', 
    'hisdx_cough_persist', 
    'hisdx_malaria_sev', 
    'hisdx_measles',
    'hydration_eye',
    'lab_urine_pos',
    'malaria_zone',
    'ear_pain_prolonged', 
    'ear_tender_swelling',
    'mouth_ds',
    'allergy_peni',
    'diarrhoea_prolonged',
    'fever_prolonged',
    'throat_assessed',
    'unclassified_eye',
    'uti_vomiting'
]

In [11]:
drop_list = [
    'agree_tx', # healthcare worker related
    'dx_agree', # healthcare worker related
    'classify_uti_complicated',
    'classify_dehydration_moderate_an',
    'classify_sam_with_complications',
    'cough_rr', # 71% nan
    'cough_oxygen', # 99% nan
    'd_nobreath_age', # 90% nan
    'd_nobreath_context', # 96% nan
    'd_nobreath_result', # 96% nan
    'abdo_ds', # 96% nan
    'adre_response', # not important
    'consultation',
    'wfa',  # 88% nan
    'wfa_less_than_neg3sd', # 64% nan
    'wfh2',  # 88% nan
    'ms_abdo_specify',  # 96% nan
    'ms_rhinitis', # 64% nan
    'hydration_sev_associated', # 96% nan
    'hydration_thirst', # 84% nan
    'hydration_neuro', # 68% nan
    'hydration_skin', # 68% nan
    'lab_hb', # 68% nan
    'labs_malaria_test', # 91% nan
    'hwrep_referral_decision', # 96%
    'sam_in_ocp', # 64% nan
    'sam_u6add_ocp', # 64% nan
    'malaria_presum', # 94% nan
    'ear_pus', # 97% nan
    'height2', # 88% nan
    'glycaemia_value', # 99% nan
    'glycaemia_unit', # 94% nan
    'anaphyl_confirmed', # 64% nan
    'high_respiratory_rate', # 64% nan
    'uvul_complication', # 64% nan
    'uvulectomy', # 64% nan
]


In [12]:
drop_semantic_list = [
    'cough_ds',
    'd_gds_colour',
    'd_gds_neuro',
    'd_gds_oth',
    'skin_symptoms',
    'skin_vesicles_loc',
    's_oth_symp_2yrs',
    's_oth_sympt_entered',
    'hwrep_comment_final',
    'hwrep_dx', # 96% nan
    'hwrep_dx_oth', # 98% nan
    'hwrep_tx', # 96% nan
    'hwrep_tx_oth', # 98% nan
    'sam_signs_complicated', # 95% nan
    'eye_ds', # 95% nan
    'eye_sympt', # 95% nan
    'vsd_added_signs', # 99% nan
    'convulsion_info', # 99% nan
    'measles_danger_signs', # 99% nan
    
]

In [13]:
df2 = df.drop(keep_list + drop_list + imbalanced_list + drop_semantic_list, axis=1)

In [14]:
df2.info(1)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 310289 entries, 0 to 310303
Empty DataFrame

In [15]:
modes = dict(df['classify_malaria'].value_counts(dropna=0))
modes

{nan: 300204, 1.0: 8204, 0.0: 1881}

In [16]:
(211898+46479+2413)/len(df2)

0.8404745253618401

In [17]:
print(modes[list(modes.keys())[0]]/len(df))

0.9674980421478041


In [18]:
modes = dict(df['ms_rhinitis'].value_counts(dropna=0))
modes[0]/(modes[0]+modes[1])

0.963386852303978

In [19]:


_, features = find_features(df2, ['classify_malaria'])
print_info(df2, features, drop_list)

In [20]:
balanced, unbalanced = balanced_features(df, features, 0.990)

In [21]:
balanced, unbalanced

([], [])

In [22]:
drop_list + imbalanced_list

['agree_tx',
 'dx_agree',
 'classify_uti_complicated',
 'classify_dehydration_moderate_an',
 'classify_sam_with_complications',
 'cough_rr',
 'cough_oxygen',
 'd_nobreath_age',
 'd_nobreath_context',
 'd_nobreath_result',
 'abdo_ds',
 'adre_response',
 'consultation',
 'wfa',
 'wfa_less_than_neg3sd',
 'wfh2',
 'ms_abdo_specify',
 'ms_rhinitis',
 'hydration_sev_associated',
 'hydration_thirst',
 'hydration_neuro',
 'hydration_skin',
 'lab_hb',
 'labs_malaria_test',
 'hwrep_referral_decision',
 'sam_in_ocp',
 'sam_u6add_ocp',
 'malaria_presum',
 'ear_pus',
 'height2',
 'glycaemia_value',
 'glycaemia_unit',
 'anaphyl_confirmed',
 'high_respiratory_rate',
 'uvul_complication',
 'uvulectomy',
 'classify_UTI_febrile',
 'classify_abdo_constipation',
 'classify_abdo_possible_strangula',
 'classify_abdo_resp_pain',
 'classify_abdo_risk_of_testicular',
 'classify_abdo_sickle_cell_crisis',
 'classify_anaphylaxis',
 'classify_anaphylaxis_non_severe',
 'classify_anaphylaxis_severe',
 'classify_anem

In [23]:
len(drop_list) + len(imbalanced_list) + len(drop_semantic_list)

179

In [24]:
df3 = df.drop(drop_list + imbalanced_list + drop_semantic_list, axis=1)

In [25]:
df3.shape, df.shape

((310289, 92), (310289, 271))

In [26]:
df3.to_csv(os.path.join(data_dir, 'processed_data.csv'), index=False)

In [27]:
df4 = pd.read_csv(os.path.join(data_dir, 'processed_data.csv'))

/Users/abdulkadirgokce/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (62,90) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [28]:
df4.info(1)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 310289 entries, 0 to 310288
Data columns (total 92 columns):
 #   Column                            Non-Null Count   Dtype  
---  ------                            --------------   -----  
 0   source                            310289 non-null  int64  
 1   project                           310289 non-null  object 
 2   hf                                310289 non-null  int64  
 3   created                           310289 non-null  object 
 4   slot_id                           310289 non-null  float64
 5   a_age                             310289 non-null  float64
 6   a_gender2                         310289 non-null  object 
 7   a_weight                          310289 non-null  float64
 8   bloody_stool                      300204 non-null  float64
 9   classify_abdo_gastro              310289 non-null  float64
 10  classify_abdo_unclassified        300204 non-null  float64
 11  classify_cough_pneumonia          310289 non-null  f